# LSTM Classification with TREC Dataset
<hr>

We will build a text classification model using LSTM model on the TREC Dataset. 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
from nltk.corpus import stopwords, twitter_samples
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold
from nltk.stem import PorterStemmer
from string import punctuation
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
import time

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

## Load the Dataset

In [2]:
corpus = pd.read_pickle('../../../0_data/TREC/TREC.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(5952, 3)


,sentence,label,split
0,how did serfdom develop in and then leave russ...,0,train
1,what films featured the character popeye doyle ?,1,train
2,how can i find a list of celebrities ' real na...,0,train
3,what fowl grabs the spotlight after the chines...,1,train
4,what is the full form of .com ?,2,train
...,...,...,...
5947,who was the 22nd president of the us ?,3,test
5948,what is the money they use in zambia ?,1,test
5949,how many feet in a mile ?,5,test
5950,what is the birthstone of october ?,1,test


In [3]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5952 entries, 0 to 5951
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  5952 non-null   object
 1   label     5952 non-null   int32 
 2   split     5952 non-null   object
dtypes: int32(1), object(2)
memory usage: 116.4+ KB


In [4]:
corpus.groupby( by=['split','label']).count()

sentence
split label          
test  0           138
      1            94
      2             9
      3            65
      4            81
      5           113
train 0          1162
      1          1250
      2            86
      3          1223
      4           835
      5           896

In [5]:
corpus.groupby(by='split').count()

,sentence,label
split,,
test,500,500
train,5452,5452


In [6]:
# Separate the sentences and the labels
# Separate the sentences and the labels for training and testing
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print(len(train_x))
print(len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print(len(test_x))
print(len(test_y))

5452
5452
500
500


# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>## Develop Vocabulary

A part of preparing text for text classification involves defining and tailoring the vocabulary of words supported by the model. **We can do this by loading all of the documents in the dataset and building a set of words.**

The larger the vocabulary, the more sparse the representation of each word or document. So, we may decide to support all of these words, or perhaps discard some. The final chosen vocabulary can then be saved to a file for later use, such as filtering words in new documents in the future.

In [7]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

print("Example of sentence: ", train_x[4])

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  what is the full form of .com ?
Into a sequence of int: [3, 4, 2, 471, 261, 5, 372]
Into a padded sequence: [  3   4   2 471 261   5 372   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [9]:
# See the first 10 words in the vocabulary

word_index = tokenizer.word_index
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
what 3
is 4
of 5
in 6
a 7
how 8
's 9
was 10
8461


# Model 1: Embedding Random
<hr>

<img src="model.png" style="width:700px;height:400px;"> <br>

## LSTM Model

In [42]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=6, activation='softmax')
    ])
    
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [43]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_28 (Bidirectio (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_29 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 6)                 774       
Total params: 586,470
Trainable params: 586,470
Non-trainable params: 0
_________________________________________________________________


In [44]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)
# callbacks = myCallback()

## Train and Test the Model

In [45]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"


# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)


# encode data using
# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)
test_sequences = tokenizer.texts_to_sequences(test_x)

max_len = max_length(training_sequences)

# Pad the sequence to have the same size
Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

word_index = tokenizer.word_index
vocab_size = len(word_index)+1

# Define the input shape
model = define_model(input_dim=vocab_size, max_length=max_len)

# Train the model
model.fit(Xtrain, train_y, batch_size=50, epochs=60, verbose=1, 
          callbacks=[callbacks], validation_data=(Xtest, test_y))

# evaluate the model
loss, acc = model.evaluate(Xtest, test_y, verbose=0)
print('Test Accuracy: {}'.format(acc*100))


Epoch 1/60
110/110 [==============================] - 47s 196ms/step - loss: 1.4642 - accuracy: 0.3943 - val_loss: 0.6511 - val_accuracy: 0.8220
Epoch 2/60
110/110 [==============================] - 15s 137ms/step - loss: 0.3279 - accuracy: 0.9124 - val_loss: 0.4272 - val_accuracy: 0.8780
Epoch 3/60
110/110 [==============================] - 14s 130ms/step - loss: 0.1063 - accuracy: 0.9735 - val_loss: 0.4817 - val_accuracy: 0.8820
Epoch 4/60
110/110 [==============================] - 16s 143ms/step - loss: 0.0375 - accuracy: 0.9899 - val_loss: 0.5721 - val_accuracy: 0.8640
Epoch 5/60
110/110 [==============================] - 15s 136ms/step - loss: 0.0255 - accuracy: 0.9951 - val_loss: 0.5366 - val_accuracy: 0.8960
Epoch 6/60
110/110 [==============================] - 14s 129ms/step - loss: 0.0070 - accuracy: 0.9983 - val_loss: 0.6522 - val_accuracy: 0.8620
Epoch 7/60
110/110 [==============================] - 15s 139ms/step - loss: 0.0046 - accuracy: 0.9994 - val_loss: 0.7505 - val_ac

## Summary

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [46]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [47]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [48]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [49]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 7526 words present from 8761 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [50]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    
    embed_matrix = np.zeros((vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.word_vec(word)
            
        # initialize the unknown word with standard normal distribution values
        else:
            embed_matrix[idx] = np.random.randn(emb_dim)
            
    return embed_matrix

In [51]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i)
em_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.54154601, -0.90240048, -0.58870309, ..., -1.41602133,
         0.47435075, -0.61934404],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## LSTM Model

In [68]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=6, activation='softmax')
    ])
    
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [69]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(vocab_size, 300))
model_0.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, 100, 300)          2538300   
_________________________________________________________________
bidirectional_48 (Bidirectio (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_49 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dropout_24 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 6)                 774       
Total params: 2,824,770
Trainable params: 286,470
Non-trainable params: 2,538,300
_________________________________________________________________


## Train and Test the Model

In [70]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [72]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"


# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)


# encode data using
# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)
test_sequences = tokenizer.texts_to_sequences(test_x)

max_len = max_length(training_sequences)

# Pad the sequence to have the same size
Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

word_index = tokenizer.word_index
vocab_size = len(word_index)+1

emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

# Define the input shape
model = define_model_2(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

# Train the model
model.fit(Xtrain, train_y, batch_size=50, epochs=60, verbose=1, 
          callbacks=[callbacks], validation_data=(Xtest, test_y))

# evaluate the model
loss, acc = model.evaluate(Xtest, test_y, verbose=0)
print('Test Accuracy: {}'.format(acc*100))


Epoch 1/60
110/110 [==============================] - 17s 71ms/step - loss: 1.5505 - accuracy: 0.3558 - val_loss: 1.0322 - val_accuracy: 0.5160
Epoch 2/60
110/110 [==============================] - 5s 48ms/step - loss: 0.7620 - accuracy: 0.7347 - val_loss: 0.5835 - val_accuracy: 0.8080
Epoch 3/60
110/110 [==============================] - 5s 47ms/step - loss: 0.4988 - accuracy: 0.8299 - val_loss: 0.5854 - val_accuracy: 0.8280
Epoch 4/60
110/110 [==============================] - 5s 47ms/step - loss: 0.4167 - accuracy: 0.8594 - val_loss: 0.5137 - val_accuracy: 0.8460
Epoch 5/60
110/110 [==============================] - 5s 47ms/step - loss: 0.2974 - accuracy: 0.9046 - val_loss: 0.4117 - val_accuracy: 0.8680
Epoch 6/60
110/110 [==============================] - 5s 50ms/step - loss: 0.2729 - accuracy: 0.9095 - val_loss: 0.4030 - val_accuracy: 0.8740
Epoch 7/60
110/110 [==============================] - 6s 53ms/step - loss: 0.2296 - accuracy: 0.9277 - val_loss: 0.4779 - val_accuracy: 0.860

## Summary

# Model 3: Word2Vec Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## LSTM Model

In [92]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=6, activation='softmax')
    ])
    
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [93]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(vocab_size, 300))
model_0.summary()

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_38 (Embedding)     (None, 100, 300)          2538300   
_________________________________________________________________
bidirectional_76 (Bidirectio (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_77 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dropout_38 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 6)                 774       
Total params: 2,824,770
Trainable params: 2,824,770
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [94]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [122]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"


# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)


# encode data using
# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)
test_sequences = tokenizer.texts_to_sequences(test_x)

max_len = max_length(training_sequences)

# Pad the sequence to have the same size
Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

word_index = tokenizer.word_index
vocab_size = len(word_index)+1

emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

# Define the input shape
model = define_model_3(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

# Train the model
model.fit(Xtrain, train_y, batch_size=50, epochs=60, verbose=1, 
          callbacks=[callbacks], validation_data=(Xtest, test_y))

# evaluate the model
loss, acc = model.evaluate(Xtest, test_y, verbose=0)
print('Test Accuracy: {}'.format(acc*100))

Epoch 1/60
110/110 [==============================] - 13s 80ms/step - loss: 1.4574 - accuracy: 0.4037 - val_loss: 0.7372 - val_accuracy: 0.7360
Epoch 2/60
110/110 [==============================] - 7s 68ms/step - loss: 0.4974 - accuracy: 0.8356 - val_loss: 0.4991 - val_accuracy: 0.8420
Epoch 3/60
110/110 [==============================] - 7s 68ms/step - loss: 0.2302 - accuracy: 0.9327 - val_loss: 0.5227 - val_accuracy: 0.8360
Epoch 4/60
110/110 [==============================] - 8s 70ms/step - loss: 0.1108 - accuracy: 0.9702 - val_loss: 0.4832 - val_accuracy: 0.8500
Epoch 5/60
110/110 [==============================] - 9s 84ms/step - loss: 0.0763 - accuracy: 0.9801 - val_loss: 0.7450 - val_accuracy: 0.8360
Epoch 6/60
110/110 [==============================] - 9s 84ms/step - loss: 0.0440 - accuracy: 0.9874 - val_loss: 0.8450 - val_accuracy: 0.8200
Epoch 7/60
110/110 [==============================] - 9s 83ms/step - loss: 0.0350 - accuracy: 0.9889 - val_loss: 0.5493 - val_accuracy: 0.872

## Summary